<a href="https://colab.research.google.com/github/Jonathan-code-hub/Many-Mini-OR-Problems/blob/main/LinearProgramming/LP_Problem_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 10: Carco uses robots to manufacture cars. The following
demands for cars must be met (not necessarily on time, but
all demands must be met by end of quarter 4): quarter 1—
600; quarter 2—800; quarter 3—500; quarter 4—400. At
the beginning of the quarter, Carco has two robots. Robots
can be purchased at the beginning of each quarter, but a
maximum of two per quarter can be purchased. Each robot
can build as many as 200 cars per quarter. It costs \$5,000
to purchase a robot. Each quarter, a robot incurs \$500 in
maintenance costs (even if it is not used to build any cars).
Robots can also be sold at the beginning of each quarter for
\$3,000. At the end of each quarter, a holding cost of \$200
per car is incurred. If any demand is backlogged, then a cost
of \$300 per car is incurred for each quarter the demand is
backlogged.

In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 95.5 MB/s eta 0:00:00


In [5]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger, value, LpStatus

# Parameters #
T = [1,2,3,4]
D = {1:600, 2:800, 3:500, 4:400}
cap = 200
buy_cost = 5000
sell_rev = 3000
maint_cost = 500
hold_cost = 200
back_cost = 300
max_buy_per_quarter = 2
R0 = 2  # robots at start before Q1 #

# Model #
m = LpProblem("Carco_Robot_Planning", LpMinimize)

# Decision vars #
buy = {t: LpVariable(f"buy_{t}", lowBound=0, upBound=max_buy_per_quarter, cat=LpInteger) for t in T}
sell = {t: LpVariable(f"sell_{t}", lowBound=0, cat=LpInteger) for t in T}
R = {t: LpVariable(f"R_{t}", lowBound=0, cat=LpInteger) for t in T}   # robots available in quarter t
P = {t: LpVariable(f"P_{t}", lowBound=0) for t in T}   # production in quarter t (cars)
I = {t: LpVariable(f"I_{t}", lowBound=0) for t in T}   # inventory end of quarter t
B = {t: LpVariable(f"B_{t}", lowBound=0) for t in T}   # backlog end of quarter t

# Objective #
m += lpSum(buy_cost * buy[t] + maint_cost * R[t] - sell_rev * sell[t] + hold_cost * I[t] + back_cost * B[t] for t in T)

# Robot balance and sale bounds #
for t in T:
    if t == 1:
        m += R[t] == R0 + buy[t] - sell[t]
        # you cannot sell more than R0 at t=1 #
        m += sell[t] <= R0
    else:
        m += R[t] == R[t-1] + buy[t] - sell[t]
        # you cannot sell more than robots you had at start of quarter t (i.e., R_{t-1}) #
        m += sell[t] <= R[t-1]

# Production capacity #
for t in T:
    m += P[t] <= cap * R[t]

# Inventory/backlog flow #
# initial I0 = B0 = 0 implicit #
for t in T:
    if t == 1:
        m += I[1] - B[1] == 0 + P[1] - D[1]
    else:
        m += I[t] - B[t] == I[t-1] - B[t-1] + P[t] - D[t]

# Must have no backlog after quarter 4 #
m += B[4] == 0

# Solve #
m.solve()

# Report results #
print("Status:", LpStatus[m.status])
print(f"Total cost = ${value(m.objective):,.2f}\n")
print("Quarter |  buy  sell   robots   prod   inv   back")
for t in T:
    print(f"{t:>3}    | {int(buy[t].value()):>4}  {int(sell[t].value()):>4}   {int(R[t].value()):>6}   "
          f"{float(P[t].value()):>6.1f}  {float(I[t].value()):>5.1f}  {float(B[t].value()):>5.1f}")



Status: Optimal
Total cost = $10,000.00

Quarter |  buy  sell   robots   prod   inv   back
  1    |    1     0        3    600.0    0.0    0.0
  2    |    1     0        4    800.0    0.0    0.0
  3    |    0     1        3    500.0    0.0    0.0
  4    |    0     1        2    400.0    0.0    0.0
